In [2]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

from keras.models import Sequential
from keras.layers import Dense, Activation, LeakyReLU
from keras import backend
from keras.callbacks import TensorBoard
from keras.optimizers import Adam

from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler, LabelBinarizer

In [3]:
df_clients = pd.read_csv("new_data/Clients_immatriculation_predicted.csv")

In [4]:
df_entrainement = df_clients[['age', 'sexe', 'taux', 'situationFamiliale','nbEnfantsAcharge', 'DeuxiemeVoiture','predicted', 'occasion']]
cols = ['age', 'taux','nbEnfantsAcharge']
mns = MinMaxScaler()
df_entrainement[cols] = MinMaxScaler().fit_transform(df_entrainement[cols])
df_market = pd.read_csv('data/Marketing.csv')
df_market[cols] = mns.fit_transform(df_market[cols])
df_ent_new = df_entrainement[df_entrainement.occasion == False]
df_ent_occ = df_entrainement[df_clients.occasion == True]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [5]:
def uniformCols(df, limit):
  df_e_s = []
  for pred in df.predicted.unique():
    df_e_s.append(df[df.predicted == pred].sample(limit))


  df_ent = pd.concat(df_e_s, ignore_index=False)

  df_ent.predicted.value_counts()


  return df

df_ent_new = uniformCols(df_ent_new, 10000)
df_ent_occ = uniformCols(df_ent_occ, 10000)

In [6]:
def dummiser(df,columns):
  for column in columns:
    print(column)
    df[pd.get_dummies(df[column]).columns] = pd.get_dummies(df[column])
    df = df.drop([column], axis=1)
  return df

def encode(df,column):
  liste = df[column].unique()
  dico = dict([(liste[i], i )for i in range(len(liste))])
  for i in df.index.tolist():
    key = df.at[i,column]
    df.at[i, column] = dico[key]
  return df[column]

def df_to_float32(df): 
  for col in df.columns.values:
    df[col] = np.asarray(df[col]).astype('float32')

def create_entree_sortie(df_ent_occ):
  entree = df_ent_occ[['age', 'sexe', 'taux', 'situationFamiliale','nbEnfantsAcharge', 'DeuxiemeVoiture']]
  entree['sexe'] = encode(entree,'sexe')
  entree['situationFamiliale'] = encode(entree,'situationFamiliale')
  entree['DeuxiemeVoiture'] = encode(entree,'DeuxiemeVoiture')
  df_to_float32(entree)

  sortie = dummiser(df_ent_occ,['predicted'])[dummiser(df_ent_occ,['predicted']).columns[-5:]]
  return entree, sortie

In [7]:
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans

In [9]:
# Voiture neuves: 
entree, sortie = create_entree_sortie(df_ent_new)
X_train, X_test, y_train, y_test = train_test_split(entree, sortie,test_size=0.02, random_state=42)

n_units = 100
layers = 3
n_batch = 100
n_epochs = 10

model = Sequential()
model.add(Dense(units = X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))

for _ in range(layers - 1):
    model.add(Dense(n_units, activation='relu'))

model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

predicted
predicted
Epoch 1/10
6565/6565 [==============================] - 15s 2ms/step - loss: 0.4920 - accuracy: 0.7587 - val_loss: 0.3497 - val_accuracy: 0.8061
Epoch 2/10
6565/6565 [==============================] - 14s 2ms/step - loss: 0.3556 - accuracy: 0.8055 - val_loss: 0.3461 - val_accuracy: 0.8116
Epoch 3/10
6565/6565 [==============================] - 14s 2ms/step - loss: 0.3519 - accuracy: 0.8062 - val_loss: 0.3421 - val_accuracy: 0.8122
Epoch 4/10
6565/6565 [==============================] - 14s 2ms/step - loss: 0.3490 - accuracy: 0.8072 - val_loss: 0.3413 - val_accuracy: 0.8118
Epoch 5/10
6565/6565 [==============================] - 14s 2ms/step - loss: 0.3469 - accuracy: 0.8080 - val_loss: 0.3433 - val_accuracy: 0.8092
Epoch 6/10
6565/6565 [==============================] - 14s 2ms/step - loss: 0.3474 - accuracy: 0.8076 - val_loss: 0.3426 - val_accuracy: 0.8074
Epoch 7/10
6565/6565 [==============================] - 14s 2ms/step - loss: 0.3460 - accuracy: 0.8065 - val_l

In [11]:
#Matrice de confusion voiture neuve

from sklearn.metrics import confusion_matrix
y_testArr = np.argmax(y_test.values, axis=1)
y_pred = model.predict(X_test)
y_pred2 = pd.DataFrame(np.argmax(y_pred, axis=1))
pd.DataFrame(confusion_matrix(y_testArr, y_pred2, normalize='true') * 100)

,0,1,2,3,4
0,60.257044,0.247158,39.396935,0.049432,0.049432
1,0.000000,99.853944,0.097371,0.048685,0.000000
2,0.000000,26.910710,73.089290,0.000000,0.000000
3,0.000000,0.000000,0.043066,66.968131,32.988803
4,0.032362,0.064725,0.000000,1.294498,98.608414


In [13]:
# Voiture occasion: 

entree, sortie = create_entree_sortie(df_ent_occ)
X_train, X_test, y_train, y_test = train_test_split(entree, sortie,test_size=0.02, random_state=42)

n_units = 100
layers = 3
n_batch = 100
n_epochs = 10

model = Sequential()
model.add(Dense(units = X_train.shape[1], input_dim=X_train.shape[1], activation='relu'))

for _ in range(layers - 1):
    model.add(Dense(n_units, activation='relu'))

model.add(Dense(5, activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train, 
                    batch_size=n_batch, epochs=n_epochs, 
                    validation_split = 0.01,
                    callbacks=[TensorBoard()],
                    verbose=1)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stab

predicted
predicted
Epoch 1/10
2975/2975 [==============================] - 7s 2ms/step - loss: 0.5734 - accuracy: 0.7312 - val_loss: 0.3637 - val_accuracy: 0.7913
Epoch 2/10
2975/2975 [==============================] - 6s 2ms/step - loss: 0.3478 - accuracy: 0.8074 - val_loss: 0.3558 - val_accuracy: 0.8040
Epoch 3/10
2975/2975 [==============================] - 6s 2ms/step - loss: 0.3443 - accuracy: 0.8069 - val_loss: 0.3545 - val_accuracy: 0.7987
Epoch 4/10
2975/2975 [==============================] - 6s 2ms/step - loss: 0.3419 - accuracy: 0.8092 - val_loss: 0.3512 - val_accuracy: 0.7953
Epoch 5/10
2975/2975 [==============================] - 6s 2ms/step - loss: 0.3406 - accuracy: 0.8102 - val_loss: 0.3495 - val_accuracy: 0.7990
Epoch 6/10
2975/2975 [==============================] - 6s 2ms/step - loss: 0.3395 - accuracy: 0.8096 - val_loss: 0.3524 - val_accuracy: 0.7907
Epoch 7/10
2975/2975 [==============================] - 6s 2ms/step - loss: 0.3388 - accuracy: 0.8097 - val_loss: 0.

In [14]:
#Matrice de confusion voiture occasion

y_testArr = np.argmax(y_test.values, axis=1)
y_pred = model.predict(X_test)
y_pred2 = pd.DataFrame(np.argmax(y_pred, axis=1))
pd.DataFrame(confusion_matrix(y_testArr, y_pred2, normalize='true') * 100)

,0,1,2,3,4
0,58.815789,0.000000,41.052632,0.00000,0.131579
1,0.000000,86.825397,12.857143,0.00000,0.317460
2,0.060533,12.227603,87.711864,0.00000,0.000000
3,0.000000,0.000000,0.000000,36.38171,63.618290
4,0.000000,0.054735,0.054735,0.00000,99.890531
